In [1]:
# İmports
import requests
from bs4 import BeautifulSoup
from IPython.display import display, Markdown
import ollama


In [2]:
MODEL = "llama3.2"


In [3]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup=BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="/n",strip=True)  # Limit to first 2000 characters    

In [5]:
deponext=Website("https://deponext.com.tr/")
print(deponext.title)
print(deponext.text[:2000])

Anasayfa | Depo Next - Güvenli Depolama Çözümleri
Eşyalarınız İçin Güvenli ve Kolay Depolama Çözümü/nKendinize Özel Depolama Alanınızı Oluşturun/nTeklif Al/nDepoNext/nKişiye Özel Depo Kiralama/n7/24/nAşama 1/nBeklentiler dinlenir,/noperasyon hakkında bilgilendirilir./nDepolanacak eşya veya ürünlerin detayına göre operasyon hakkında bilgilendirilir. Eşya veya ürünlerin nasıl taşınacağı, depolama alanı gibi detaylar konuşulur/nAşama 2/nİhtiyaca uygun,/nteklif verilir./nDepolanacak eşyaların veya ürünlerin durumu göz önünde bulundurularak teklif verilir. Teklifte hem depo ücreti hem de taşıma ücreti belirtilir/nAşama 3/nEşyalar taşınmaya ve,/ndepolanmaya hazır./nEşyalar belirtildiği gibi paketlenir ve depoya taşınır. Eşyaların depolanma sürecinde paketleme malzemeleri eşyaların üzerinde kalır./nAşama 4/nEşyalarınız size özel,/noda deponuzda güvende./nEşyalarınızı görebileceğiniz, size özel bir alana yerleştirildi ve güvende. İçiniz rahat olabilir. Konteyner çözümünde de size özel bir alan

In [6]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:

# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
def summarize_website(url):
    website=Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    display(Markdown(response['message']['content']))

In [10]:
summarize_website("https://deponext.com.tr/")

**Depo Next | Güvenli Depolama Çözümleri**
=============================================

Depo Next, Türkiye'de güvenli ve Kolay Depolama Çözümleri sunan bir şirketdir. Şirket, personele özel depolama alanını oluşturmak ve eşyaları güvenli ve kolayca depolamak isteyen kişilere hizmet vermektedir.

**Hizmetler**
------------

*   Personelene Özel Depolama Alanı
*   7/24 Hizmet
*   Teklif Alma
*  Depolanacak Eşyanın Detayına Göre Operasyon Haklı
*   İhtiyaca Uygunning, Tökelif Verilmesi
*   Eşyaların Demontesi, Paketlenmesi, Taşınma ve Depolanması

**Müşteri Đeğrilikleri**
-------------------------

*   Müşteri Memnuniyeti %98
*   Şehirleriçi Hizmet
*   Aktif Depo Alanı 12.000 m²
*   Mühendislik ve İşletme Faaliyetleri

**İletişim**
-------------

*   Address: Gayrettepe, Vefabey Sokağı 25/1, 34349 Beşiktaş/İstanbul
*   Phone: +90 537 323 43 43
*   Email: info@deponext.com

In [11]:
def display_summary(url):
    summary=summarize_website(url)
    display(Markdown(summary))

In [12]:
display_summary("https://deponext.com.tr/")

**Güvenli Depolama Çözümleri - DepoNext**

DpoNext, Türkiye'de binlerce kişi ve işletmeye güvenilen güvenli depolama ve taşıma hizmeti sunuyor. Şikayetlerin %98'i memnun kalınan service dengan modern konteyner çözümünde size özel alanlara yerendir.

**Kriterler:**

- **Güvenli**: Müşterilerimizin güvenini kazandık.
- **Modern Konteyner Çözümü**: Modern konteyner çözümü ile eşylerin demontesi, paketlenmesi, taşınması ve depolanması.
- **Etkin Service**: Her aşama boyunca bilgilendirildik. Ekipin ilgisi ve profesyonelliği harika oldu.
- **Temiz ve Güvenli Alanlar**: Depolama alanları sạch ve güvenlidir.

**Tavsiye:**

DepoNext sizler için her şeyi düşündük. Siz rahat bir nefes alabilirsiniz. Her şeyden daha fazlasını öğrenin!

<IPython.core.display.Markdown object>